**Spark 编程进阶**

本章介绍两种类型的共享变量：**累加器(accumulator)** 与 **广播变量(broadcast variable)**。

累加器用来对信息进行聚合，广播变量用来高效分发比较大的对象。

通常在向 Spark 传递函数时，比如使用 `map()` 或者 `filter()` 传条件时，可以使用驱动器程序中定义的变量，但是集群中运行的每个任务都会得到这些变量的一份新的副本，更新副本的值不会改变驱动器中的对应变量。Spark 的两个共享变量——**累加器与广播变量**——分别为结果聚合与广播这两种常见的通信模式突破这一限制。

In [109]:
import findspark
# 找到spark的安装目录
spark_path = 'C:\spark232_hadoop27'
findspark.init(spark_path,edit_rc=True)
import pyspark
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('Chen_App')
try:
    sc.stop()
except:
     print('there is no sparkcntext on running')   
sc = SparkContext(conf=conf)

# 累加器

Spark 的第一种共享变量就是**累加器**，累加器提供了将工作节点中的值聚合到驱动器程序中的简单语法。累加器的一个常见用途在调试时对作业执行过程中的事件进行**计数**。

### class pyspark.Accumulator()

`class pyspark.Accumulator(aid, value, accum_param)`:

A shared variable that can be accumulated, i.e., has a commutative and associative “add” operation. Worker tasks on a Spark cluster can add values to an Accumulator with the **+=** operator, but only the driver program is allowed to access its **value**, using value. Updates from the workers get propagated automatically to the driver program.

While **SparkContext** supports accumulators for primitive data types like **int** and **float**, users can also define accumulators for custom types by providing a custom **AccumulatorParam** object.Refer to the doctest of this module for an example.

* `add(term)`:Adds a term to this accumulator’s value

* `value`:Get the accumulator’s value; only usable in driver program

注意：工作节点上的任务不能访问累加器的值。从这些任务角度看，**累加器是一个只写变量。**

在这种模式下，累加器的实现可以更加高效，不需要对每次更新操作进行复杂的通信。

### accumulator()

`accumulator(value, accum_param=None)`:

Create an **Accumulator** with the given initial value, using a given **AccumulatorParam** helper object to define how to add values of the data type if provided. Default AccumulatorParams are used for **integers and floating-point** numbers if you do not provide one. For other types, a custom AccumulatorParam can be used.

注：在刚开始启动 SparkContext 时，`accumulator()` 属于 `class pyspark.SparkContext()` 下面的一个方法，所以不用单独导入`from pyspark import Accumulator` 就可以使用。

In [110]:
acc=sc.accumulator(0)
acc.add(9)
print('after add op value is:{}'.format(acc.value))
acc += 1
print('after += op value is:{}'.format(acc.value))

after add op value is:9
after += op value is:10


### class pyspark.AccumulatorParam

`class pyspark.AccumulatorParam
`:Helper object that defines how to accumulate values of a given type.

* `addInPlace(value1, value2)`:Add two values of the accumulator’s data type, returning a new value; for efficiency, can also update value1 in place and return it.

* `zero(value)`:Provide a “zero value” for the type, compatible in dimensions with the provided value (e.g., a zero vector)

In [111]:
from pyspark import AccumulatorParam

# 自定义一个累加器，对tuple 的每一项进行求和
class StringAndIntAccumulator(AccumulatorParam):
    def zero(self, s):
        return (s[0]+'chenbofeng_',0)
    def addInPlace(self, s1, s2):
        return (s1[0]+s2[0], s1[1]+s2[1])

accumulator = sc.accumulator(("",0), StringAndIntAccumulator())

def add(x): 
    global accumulator
    accumulator += x

sc.parallelize([("a",1), ("b",2), ("c",3)]).foreach(add)

accumulator.value

('chenbofeng_abc', 6)

在上面这个例子中，`accumulator` 有自己的两个属性`add`以及`value`，但是`StringAndIntAccumulator`继承了类`AccumulatorParam`的两个方法`zero`和`addInPlace`，重新定义了 `add` 和 `zero`。

# 广播变量

Spark 的第一种共享变量就是**广播变量**，它可以让程序高效地向所有工作节点发送一个较大的只读值，以供一个或多个 Spark 操作使用。比如如果应用需要向所有节点发送一个较大的只读查询表或者机器学习算法中的一个很大的特征向量，广播变量都会是很好地选择。

使用广播变量的方式很简单：

* 通过一个类型 T 的对象调用 `SparkContext.broadcast` 创建一个 `Broadcast[T]` 对象。任何序列化的类型都可以这么实现；


* 通过 `value` 属性访问该对象的值；


* 变量只会被发到各个节点一次，应作为**只读值**处理，修改这个值不会影响到别的节点。

    满足**只读**要求最容易的方式是广播基本类型的值或者引用不可变对象；但有时候传输一个可变对象可能更为方便高效，如果这样的话，需要自己维护只读的条件。

### class pyspark.Broadcast()

`class pyspark.Broadcast(sc=None, value=None, pickle_registry=None, path=None, sock_file=None)`:A broadcast variable created with `SparkContext.broadcast()`. Access its value through `value`.

In [112]:
b = sc.broadcast([1, 2, 3, 4, 5])
b.value

[1, 2, 3, 4, 5]

### broadcast()

`broadcast(value)`:Broadcast a **read-only variable** to the cluster, returning a `L{Broadcast<pyspark.broadcast.Broadcast>}` object for reading it in distributed functions. The variable will be sent to each cluster only once.

# 基于分区进行操作

基于分区对数据进行操作可以让我们避免为每个数据元素进行重复的配置工作。诸如打开数据库连接或创建随机数生成器等操作，都是我们应当尽量避免为每个元素都配置一次的工作。

Spark 提供基于分区的 `map` 和 `foreach`，让你的部分代码只对 RDD 的每个分区运行一次，这样可以帮助降低这些操作的代价。

### mapPartitions()

`mapPartitions(f, preservesPartitioning=False)`:Return a new RDD by applying a function to each partition of this RDD.

* 该函数和map函数类似，只不过映射函数的参数由RDD中的每一个元素变成了RDD中每一个分区的迭代器。如果在映射的过程中需要频繁创建额外的对象，使用`mapPartitions`要比`map`高效；

    比如，将RDD中的所有数据通过JDBC连接写入数据库，如果使用map函数，可能要为每一个元素都创建一个connection，这样开销很大，如果使用`mapPartitions`，那么只需要针对每一个分区建立一个connection。


* 参数`preservesPartitioning`表示是否保留父RDD的partitioner分区信息。

In [113]:
rdd = sc.parallelize([1, 2, 3, 4], 2)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f).collect()

[3, 7]

### mapPartitionsWithIndex()

`mapPartitionsWithIndex(f, preservesPartitioning=False)`:Return a new RDD by applying a function to each partition of this RDD, while tracking the index of the original partition.

函数作用同mapPartitions，不过`f`提供了两个参数，第一个参数为分区的索引。

In [114]:
rdd = sc.parallelize([1, 2, 3, 4], 3)
def f(splitIndex, iterator): yield splitIndex
rdd.mapPartitionsWithIndex(f).sum()

3

上述 rdd 三个分区的index为 0、1、2，所以求和为 3。

### foreachPartition()

`foreachPartition(f)`:Applies a function to each partition of this RDD.

In [119]:
def f(iterator):
    for x in iterator:
        x+1
sc.parallelize([1, 2, 3, 4, 5]).foreachPartition(f)